In [1]:
import sqlite3
from pathlib import Path

import pandas as pd
from gf_utils.stc_data import get_stc_data

from event_record import EventRecord

gamedata = get_stc_data(
    ["../GF_Data_Tools/data/ch/stc", "../GF_Data_Tools/data/ch/catchdata"],
    "../GF_Data_Tools/data/ch/asset/table",
)
eventrecord = EventRecord("./events.hjson")
path = Path("../Elisa/logs/develop_log.db").resolve()
conn = sqlite3.connect(f"file://{path}?immutable=1", uri=True)

gun_nm = pd.read_sql("select * from gun_nm", conn)

In [2]:
def analyze_gun_nm(full_record: pd.DataFrame):
    formula_series = {}
    for formula in gamedata["recommended_formula"].values():
        if formula["develop_type"] != 1 or formula["name"] == "人形混合":
            continue
        mp, ammo, mre, part = formula['mp'],formula['ammo'],formula['mre'],formula['part']
        record = full_record.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
        r = record.query(f"trust_2==True")["gun_rank"].value_counts(normalize=True)
        formula_series[((mp,ammo,mre,part),"rank","总数据")] = len(record.query(f"trust_2==True"))
        for i in range(2, 6):
            formula_series[((mp,ammo,mre,part),"rank",i)] = r[i]
        for i in range(2, 6):
            rank_record = record.query(f"gun_rank==@i and trust_{i}==True")
            formula_series[((mp,ammo,mre,part),f"rank{i}", "总数据")] = len(rank_record)
            res = rank_record["gun_id"].value_counts(normalize=True)
            for k, v in res.items():
                formula_series[((mp,ammo,mre,part),f"rank{i}", gamedata["gun"][k]["name"])] = v
    d = pd.Series(formula_series)
    return d

        # p = 1
        # s = {i: p - (p := p - p * r[i]) for i in range(2, 6)}
        # record = record.query(
        #     " or ".join([f"(gun_rank=={i} and trust_{i}==True)" for i in range(2, 6)])
        # )
        # record: pd.Series = record.groupby(["mp", "ammo", "mre", "part", "gun_rank"])[
        #     "gun_id"
        # ].value_counts(normalize=True)
        # for i in range(2, 6):
        #     record[record.index.get_level_values("gun_rank") == i] *= s[i]
        # formula_series.append(record)
    # return pd.concat(formula_series)

In [6]:
event_df = []
final_note = None
for note, time_query in eventrecord["gun_nm"]:
    full_records = gun_nm.query(time_query)
    # print(note, time_query)
    if len(full_records) == 0:
        continue
    final_note = note
    r = analyze_gun_nm(full_records)
    event_df.append(r.rename(note))
event_df = pd.concat(event_df,axis=1).fillna(0)

In [7]:
event_df.index.rename(["formula","type","target"], inplace=True)

In [8]:
event_df.sort_values(final_note, inplace=True)
event_df.sort_index(kind="stable", key=lambda col: col.map(lambda x:0) if col.name=="target" else col, inplace=True)

In [9]:
event_df

开始记录         日常（玛姬）     人形UP（玛姬）  \
formula              type  target                                             
(100, 400, 400, 200) rank  5           0.083387       0.089515     0.079481   
                           4           0.166452       0.170586     0.153895   
                           3           0.198747       0.197318     0.194110   
                           2           0.551414       0.542582     0.572514   
                           总数据     12448.000000  123410.000000  6316.000000   
...                                         ...            ...          ...   
(800, 800, 100, 400) rank5 MG5         0.124277       0.119824     0.052632   
                           MG36        0.075145       0.089427     0.210526   
                           PKP         0.072254       0.091189     0.000000   
                           88式         0.164740       0.125110     0.157895   
                           总数据       346.000000    2270.000000    19.000000   

                                       圣建（23春节）       日常（M110）    人形UP（M110）  \
formula              type  target                                              
(100, 400, 400, 200) rank  5           0.170256       0.089511      0.086175   
                           4           0.316584       0.179537      0.192530   
                           3           0.138597       0.200713      0.197831   
                           2           0.374563       0.530239      0.523464   
                           总数据     16299.000000  110835.000000  33200.000000   
...                                         ...            ...           ...   
(800, 800, 100, 400) rank5 MG5         0.145022       0.117224      0.110429   
                           MG36        0.067100       0.087177      0.073620   
                           PKP         0.127706       0.079560      0.042945   
                           88式         0.095238       0.106644      0.141104   
                           总数据       462.000000    2363.000000    163.000000   

                                      日常（30200）      圣建（23周年）   日常（QBU191）  
formula              type  target                                           
(100, 400, 400, 200) rank  5           0.088318      0.167615     0.095281  
                           4           0.181641      0.323955     0.164719  
                           3           0.197615      0.137583     0.190787  
                           2           0.532427      0.370846     0.549213  
                           总数据     56342.000000  18447.000000  4450.000000  
...                                         ...           ...          ...  
(800, 800, 100, 400) rank5 MG5         0.128958      0.135534     0.103448  
                           MG36        0.087507      0.086881     0.137931  
                           PKP         0.087507      0.085143     0.172414  
                           88式         0.119171      0.129453     0.172414  
                           总数据      1737.000000   1151.000000    29.000000  

[401 rows x 9 columns]

In [71]:
event_df.to_excel("gun_nm.xlsx")